#### 01 - Ingest Azure Compute Documentation into databricks Unity Catalog

This notebook downloads Azure Compute documentation from GitHub, cleans Markdown content, and writes the data into a Unity Catalog–managed Delta table.

**Execution environment**
- Run this notebook on **Azure Databricks (Premium tier)** with **Unity Catalog enabled**
- Uses a **single-user Databricks cluster** (DBR 15+)
- Writes data as **Unity Catalog–managed Delta tables**

**What this notebook does**
- Downloads the latest Azure Compute documentation from the public GitHub repository  
  `MicrosoftDocs/azure-compute-docs` using a shallow Git clone
- Parses and cleans Markdown files under the `articles/` directory
- Extracts metadata such as document ID, category, title, source URL, and ingestion time
- Persists the processed documents into a governed Delta table: databricks_rag_demo.default.raw_azure_compute_docs

This notebook establishes the **raw document ingestion layer** for a Retrieval-Augmented Generation (RAG) pipeline and intentionally avoids legacy DBFS-based storage in favor of **Unity Catalog–managed data objects**.


In [0]:
%run ./00_constants

In [0]:
%sql

----- workspace will create a default catelog with same name as the workspace, we will mostly work in this catelog
SHOW CATALOGS;

catalog
databricks_rag_demo
samples
system


In [0]:
%sql SHOW SCHEMAS IN databricks_rag_demo;

databaseName
default
information_schema


In [0]:
import os
import re
from pathlib import Path
from pyspark.sql import Row
from datetime import datetime
import shutil
import stat
from collections import defaultdict

In [0]:
TARGET_ARTICLE_PATH = f"{DOC_LOCAL_DOWNLOAD_DIR}/articles"

In [0]:
def download_azure_compute_docs():

    ## delete existing download
    if os.path.exists(DOC_LOCAL_DOWNLOAD_DIR):
        print(f"Deleting {DOC_LOCAL_DOWNLOAD_DIR}")
        shutil.rmtree(DOC_LOCAL_DOWNLOAD_DIR)
    else:
        print(f"Folder does not exist: {DOC_LOCAL_DOWNLOAD_DIR}")

    try:
        import git
    except ImportError:
        # gitpython is a Python wrapper around the git command. We will use this to do git clone
        %pip install gitpython
        dbutils.library.restartPython()
    
    from git import Repo

    # SHALLOW clone
    Repo.clone_from(
        DOC_REPO_URL,
        DOC_LOCAL_DOWNLOAD_DIR,
        depth=1 # Depth = how much git history you download
    )

download_azure_compute_docs()
os.listdir(f"{TARGET_ARTICLE_PATH}")

Folder does not exist: /tmp/azure_compute_docs


['container-instances',
 'virtual-machines',
 'virtual-machine-scale-sets',
 'azure-impact-reporting',
 'service-fabric']

In [0]:
# Function to clean markdown text
def clean_markdown(md_text: str) -> str:
    # Remove code blocks
    #md_text = re.sub(r"```.*?```", "", md_text, flags=re.S)
    # Remove images
    md_text = re.sub(r"!\[.*?\]\(.*?\)", "", md_text)
    # Remove links but keep text
    md_text = re.sub(r"\[(.*?)\]\(.*?\)", r"\1", md_text)
    # Remove headings symbols
    md_text = re.sub(r"#+ ", "", md_text)
    return md_text.strip()

In [0]:
def prepare_spark_row_data():
    article_path = Path(TARGET_ARTICLE_PATH)
    rows = []
    min_length = 500  # skip stubs / TOCs

    counts = defaultdict(int)

    for md_file in article_path.rglob("*.md"):
        try:
            with open(md_file, "r", encoding="utf-8", errors="ignore") as f:
                raw_md = f.read()

            cleaned = clean_markdown(raw_md)
            if len(cleaned) < min_length:
                continue

            rel_path = str(md_file.relative_to(article_path))
            category = rel_path.split("/", 1)[0] if "/" in rel_path else rel_path

            #### Limit the docs used for each category, you can comment out if want to use all docs
            if counts[category] >= MAX_DOCS_PER_CATEGORY:
                continue

            rows.append(Row(
                doc_id=rel_path,
                source="azure-compute-docs",
                category=category,  # e.g. virtual-machines
                title=md_file.stem,
                raw_text=cleaned,
                url=f"https://github.com/MicrosoftDocs/azure-compute-docs/blob/main/articles/{md_file.relative_to(TARGET_ARTICLE_PATH)}",
                ingest_time=datetime.utcnow()
            ))

            counts[category] += 1

        except Exception as e:
            # Fail-safe: skip bad files
            continue
    return rows

rows = prepare_spark_row_data()
print(f"Found {len(rows)} docs")
docs_df = spark.createDataFrame(rows)

display(docs_df.limit(2))



/home/spark-0d3d34eb-5e02-4fb8-ae2d-f1/.ipykernel/2179/command-8004489376597638-1726442655:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ingest_time=datetime.utcnow()


Found 214 docs


doc_id source category title raw_text url ingest_time container-instances/container-instances-quickstart.md azure-compute-docs container-instances container-instances-quickstart ---
title: Quickstart - Deploy Docker container to container instance - Azure CLI
description: In this quickstart, you use the Azure CLI to quickly deploy a containerized web app that runs in an isolated Azure container instance
ms.topic: quickstart
ms.author: tomcassidy
author: tomvcassidy
ms.service: azure-container-instances
services: container-instances
ms.date: 11/17/2025
ms.update-cycle: 180-days
ms.custom: mvc, devx-track-azurecli, mode-api
Customer intent: As a developer, I want to quickly deploy a Docker container using the command line, so that I can run my web application without managing complex orchestration platforms.
---

Quickstart: Deploy a container instance in Azure using the Azure CLI

Use Azure Container Instances to run serverless Docker containers in Azure with simplicity and speed. Deploy an application to a container instance on-demand when you don't need a full container orchestration platform like Azure Kubernetes Service.

In this quickstart, you use the Azure CLI to deploy an isolated Docker container and make its application available with a fully qualified domain name (FQDN). A few seconds after you execute a single deployment command, you can browse to the application running in the container:

![View an app deployed to Azure Container Instances in browser][aci-app-browser]

!INCLUDE [quickstarts-free-trial-note]

!INCLUDE [azure-cli-prepare-your-environment.md]

- This quickstart requires version 2.0.55 or later of the Azure CLI. If using Azure Cloud Shell, the latest version is already installed.

 > [!WARNING]
 > Best practice: User’s credentials passed via command line interface (CLI) are stored as plain text in the backend. Storing credentials in plain text is a security risk; Microsoft advises customers to store user credentials in CLI environment variables to ensure they are encrypted/transformed when stored in the backend. 

Create a resource group

Azure container instances, like all Azure resources, must be deployed into a resource group. Resource groups allow you to organize and manage related Azure resources.

First, create a resource group named *myResourceGroup* in the *eastus* location with the [az group create][az-group-create] command:

```azurecli-interactive
az group create --name myResourceGroup --location eastus
```

Create a container

Now that you have a resource group, you can run a container in Azure. To create a container instance with the Azure CLI, provide a resource group name, container instance name, and Docker container image to the [az container create][az-container-create] command. In this quickstart, you use the public `mcr.microsoft.com/azuredocs/aci-helloworld` image. This image packages a small web app written in Node.js that serves a static HTML page.

You can expose your containers to the internet by specifying one or more ports to open, a DNS name label, or both. In this quickstart, you deploy a container with a DNS name label so that the web app is publicly reachable.

Execute a command similar to the following to start a container instance. Set a `--dns-name-label` value that's unique within the Azure region where you create the instance. If you receive a "DNS name label not available" error message, try a different DNS name label.

```azurecli-interactive
az container create --resource-group myResourceGroup --name mycontainer --image mcr.microsoft.com/azuredocs/aci-helloworld --dns-name-label aci-demo --ports 80 --os-type linux --memory 1.5 --cpu 1
```

To deploy the container into a specific availability zone, use the `--zone` argument and specify the logical zone number:

```azurecli-interactive
az container create --resource-group myResourceGroup --name mycontainer --image mcr.microsoft.com/azuredocs/aci-helloworld --dns-name-label aci-demo --ports 80 --os-type linux --memor

In [0]:
# In Databricks notebooks, spark is: a pre-initialized SparkSession object that Databricks creates for you.

type(spark)

pyspark.sql.connect.session.SparkSession

In [0]:
# write to table

(
    docs_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(RAW_DOCS_TABLE)
)

In [0]:
## Validate data is stored into the table successfully

spark.sql(f"""
    SELECT COUNT(*) FROM {RAW_DOCS_TABLE}
""").display()

count(1)
214


In [0]:
## delete this table if needed, this will clean up the environment

# spark.sql(f"""
#     DROP TABLE {RAW_DOCS_TABLE}
# """).display()